The general goal of this notebook is to develop pre-processing scripts that aim to predict percentiles of the outputs's unconditional PDF rather than physical values.

# Imports

In [ ]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
#import tensorflow.math as tfm
from tensorflow import math as tfm
#import tensorflow_probability as tfp
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
#import cartopy.crs as ccrs
import matplotlib.ticker as mticker
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle
import sklearn
from sklearn.linear_model import LinearRegression
from scipy import stats
import scipy.integrate as sin

In [ ]:
fz = 15
lw = 4
siz = 100
S0 = 320 # Representative mean solar insolation for normalization purposes
S0max = 1410.6442 # Max solar insolation for normalization purposes
SN = S0/100 # Representative target = mean insolation / 4
XNNA = 1.25 # Abscissa where architecture-constrained network will be placed
XTEXT = 0.25 # Text placement
YMIN = -1 # Representative value for conserving network
YTEXT = 0.3 # Text placement

In [ ]:
plt.rc('text', usetex=False)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
plt.rc('font', family='serif', size=fz)
mpl.rcParams['lines.linewidth'] = lw
plt.close('all')

In [ ]:
import psutil
psutil.virtual_memory()

# Load training set in memory

## Load data

In [ ]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

In [ ]:
path_TRAIN = path_data + '2021_01_24_O3_TRAIN_shuffle.nc'

In [ ]:
path_TRAIN

In [ ]:
train_data = xr.open_dataset(path_TRAIN)

In [ ]:
train_data

In [ ]:
train_data['var_names']

In [ ]:
train_data['vars'].shape

In [ ]:
train_data['var_names'][-120:]

## Create histogram bins based on max,min

In [ ]:
train_data_max = train_data['vars'].max(axis=0)

In [ ]:
train_data_min = train_data['vars'].min(axis=0)

In [ ]:
train_data_max[-121]

In [ ]:
train_data_min[-121]

In [ ]:
train_data_max.shape

In [ ]:
214-121

In [ ]:
train_data_max[94]

In [ ]:
train_data_max[93]

In [ ]:
plt.plot(train_data_min[-120:])
plt.plot(train_data_max[-120:])

In [ ]:
train_data_bins = np.linspace(start=train_data_min,stop=train_data_max,num=1000)

In [ ]:
train_data_bins.shape

In [ ]:
train_data_min[-91]

In [ ]:
train_data_max[-91]

## Calculate quantile for each output variable

In [ ]:
train_data['vars'][:,-91].quantile(np.array([0,1])).values

In [ ]:
quantile_array = np.linspace(0,1,1001)

In [ ]:
PERC_array = np.zeros((1001,184))

In [ ]:
for ivar in range(214):
    print('ivar=',ivar,'           ',end='\r')
    PERC_array[:,ivar] = train_data['vars'][:,ivar].quantile(quantile_array).values

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

hf = open(pathPKL+'2021_01_24_O3_TRAIN_shuffle'+'_PERC.pkl','wb')

F_data = {'quantile_array':quantile_array,'PERC_array':PERC_array}

pickle.dump(F_data,hf)
hf.close()

In [ ]:
F_data

In [ ]:
F_data['PERC_array'][:,-91]

## Repeat for every dataset

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

In [ ]:
quantile_array = np.linspace(0,1,1001)

In [ ]:
path_array = [
    '2021_06_03_RG_TRAIN_shuffle.nc',
    '2021_06_03_RG_VALID.nc',
    '2021_06_06_RG_TEST.nc'
#    '2021_04_18_RG_TRAIN_M4K_shuffle.nc',
#     '2021_04_18_RG_VALID_M4K.nc',
#     '2021_04_18_RG_TEST_M4K.nc',
#     '2021_04_18_RG_TRAIN_P4K_shuffle.nc',
#     '2021_04_18_RG_VALID_P4K.nc',
#     '2021_04_18_RG_TEST_P4K.nc'
]

In [ ]:
#path_array = [
#     '2021_03_18_O3_TRAIN_M4K_shuffle.nc',
#     '2021_03_18_O3_VALID_M4K.nc',
#     '2021_03_18_O3_TEST_P4K.nc',
#     '2021_01_24_O3_TRAIN_shuffle.nc',
#     '2021_03_18_O3_TEST_M4K.nc',
#     '2021_01_24_O3_VALID.nc',
#     '2021_01_24_O3_TEST.nc',
#     '2021_03_18_O3_TRAIN_P4K_shuffle.nc',
#    '2021_03_18_O3_VALID_P4K.nc'
#]

In [ ]:
Ninput = 184

In [ ]:
Ninput

In [ ]:
path_array

In [ ]:
for ipath,path in enumerate(path_array):
    print('ipath=',ipath,'path=',path,'     ',end='\n')
    
    path_var = path_data + path 
    train_data = xr.open_dataset(path_var)
    
    PERC_array = np.zeros((1001,Ninput))
    
    for ivar in range(Ninput):
        print('ivar=',ivar,'           ',end='\r')
        PERC_array[:,ivar] = train_data['vars'][:,ivar].quantile(quantile_array).values
    
    hf = open(pathPKL+'/'+path+'_PERC.pkl','wb')

    F_data = {'quantile_array':quantile_array,'PERC_array':PERC_array}

    pickle.dump(F_data,hf)
    hf.close()
    print('\n')

# Test interpolation onto percentile array

Directly test in preprocessing mode to facilitate duplication of training/validation/test in percentile space

## Testing different interpolation functions

### Imports

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

In [ ]:
pathPKL_train = pathPKL + '/PKL_DATA2021_03_18_O3_TRAIN_M4K_shuffle.nc_PERC.pkl'
pathPKL_test = pathPKL + '/PKL_DATA2021_03_18_O3_TEST_P4K.nc_PERC.pkl'

In [ ]:
hf = open(pathPKL_train,'rb')
tmp = pickle.load(hf)
PERC_array = tmp['PERC_array']
quantile_array = tmp['quantile_array']

In [ ]:
PERC_array.shape

### Load small data

In [ ]:
path_dm4K = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.'
ds = xr.open_mfdataset(path_dm4K+'0001-01-05-00000.nc',\
                          decode_times=False)

In [ ]:
var0 = ds['PHQ'][1:]

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import scipy

class scipy.interpolate.interp1d(x, y, kind='linear', axis=- 1, copy=True, bounds_error=None, fill_value=nan, assume_sorted=False)[source]

In [ ]:
var_interp = interp1d(x=PERC_array[:,-91],y=quantile_array)

In [ ]:
scale_dict = load_pickle('/export/home/tbeucler/CBRAIN-CAM/nn_config/scale_dicts/009_Wm2_scaling.pkl')

In [ ]:
scale_dict['PHQ']

In [ ]:
plt.hist(scale_dict['PHQ'][-1]*PERC_array[:,-91],bins=100);

In [ ]:
var0.shape

In [ ]:
var0_afterinterp = var_interp(var0[:,-1,:,:])

In [ ]:
var0_afterinterp.shape

In [ ]:
plt.hist(var0_afterinterp.flatten(),bins=100);

In [ ]:
path_dm4K = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.'
ds = xr.open_mfdataset(path_dm4K+'0001-03-22-00000.nc',\
                          decode_times=False)

In [ ]:
plt.hist((var_interp(ds['PHQ'][:,-1,:,:])).flatten(),bins=100);

In [ ]:
var_interp = interp1d(x=PERC_array[:,-1],y=quantile_array,fill_value=(0,1))

In [ ]:
var_interp._extrapolate='True'

In [ ]:
plt.hist((var_interp(ds['QRS'][:,-1,:,:])).flatten(),bins=100);

In [ ]:
var_interp = interp1d(x=PERC_array[:,-31],y=quantile_array,fill_value=(0,1))

plt.hist((var_interp(ds['QRL'][:,-1,:,:])).flatten(),bins=100);

## Check with original dataset

In [ ]:
path_array = [
#    '2021_03_18_O3_TRAIN_M4K_shuffle.nc',
#    '2021_03_18_O3_VALID_M4K.nc',
    '2021_03_18_O3_TEST_P4K.nc',
#    '2021_01_24_O3_TRAIN_shuffle.nc',
#    '2021_03_18_O3_TEST_M4K.nc',
#    '2021_01_24_O3_VALID.nc',
#    '2021_01_24_O3_TEST.nc',
#    '2021_03_18_O3_TRAIN_P4K_shuffle.nc',
#    '2021_03_18_O3_VALID_P4K.nc'
]

In [ ]:
for ipath,path in enumerate(path_array):
    print('ipath=',ipath,'path=',path,'     ',end='\n')
    
    path_var = path_data + path 
    train_data = xr.open_dataset(path_var)
    
    hf = open(pathPKL+'/'+path+'_PERC.pkl','rb')
    tmp = pickle.load(hf)
    
    PERC_array = tmp['PERC_array']
    quantile_array = tmp['quantile_array']

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import scipy

In [ ]:
interp_fx = interp1d(x=PERC_array[:,-91],y=quantile_array)

In [ ]:
plt.hist((interp_fx(train_data['vars'][:,-91])).flatten(),bins=100);

## Develop preprocessing routine for convert_dataset_20191113

TODO = Test
def compute_perc(ds, var, PERC_array, quantile_array):
    
    from scipy.interpolate import interp1d
    
    # Load variable from dataset
    var0 = ds[var[:-4]][1:]
    # Manually entering ranges for each variable: 
    # This should change and PERC_array should be a dictionary with varianble names
    i0 = {}
    i0['PHQ'] = 94
    i0['TPHYSTND'] = 124
    i0['QRL'] = 154
    i0['QRS'] = 184
    
    # Project onto 1D percentile space to form the output
    output_percentile = 0*var0**0 # Initialization
    for ilev in range(var0.shape[1]):
        print('Interpolating level ',ilev,'out of ',var0.shape[1])
        interp_fx = interp1d(x=PERC_array[:,i0[var[:-4]]+ilev,:,:],y=quantile_array)
        output_percentile[:,ilev,:,:] = interp_fx(var0[:,ilev,:,:])
        
    return output_percentile

Load dataset

In [ ]:
path_PERC = pathPKL+'/'+'2021_03_18_O3_TRAIN_M4K_shuffle.nc'+'_PERC.pkl'

In [ ]:
path_PERC

In [ ]:
import pickle
hf = open(path_PERC,'rb')
tmp = pickle.load(hf)
PERC_array = tmp['PERC_array']
quantile_array = tmp['quantile_array']

In [ ]:
path_m4K_file = '/DFS-L/DATA/pritchard/tbeucler/SPCAM//sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.00'

In [ ]:
datam4K = xr.open_mfdataset(path_m4K_file+'01-06-1*-00000.nc')

In [ ]:
var = 'QRLPERC'
ds = datam4K

In [ ]:
var0 = ds[var[:-4]][1:]

In [ ]:
var0

In [ ]:
i0 = {}
i0['PHQ'] = 94
i0['TPHYSTND'] = 124
i0['QRL'] = 154
i0['QRS'] = 184

In [ ]:
output_percentile = 0*var0**0 # Initialization

In [ ]:
S = var0.shape[1]

In [ ]:
S

In [ ]:
ilev = 29

In [ ]:
interp_fx = interp1d(x=PERC_array[:,i0[var[:-4]]+ilev],y=quantile_array)

In [ ]:
interp_fx

In [ ]:
var0

In [ ]:
var0.shape

In [ ]:
var0[:,ilev,:,:].shape

In [ ]:
var0_interp = interp_fx(var0[:,ilev,:,:])

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))
ax[0].hist(var0[:,ilev,:,:].values.flatten(),bins=100);
ax[1].hist(PERC_array[:,i0[var[:-4]]+ilev],bins=100);
#ax[2].hist(var0_interp.flatten(),bins=100);

In [ ]:
PERC_array[:,i0[var[:-4]]+ilev]

In [ ]:
var0.shape

In [ ]:
# Project onto 1D percentile space to form the output
output_percentile = np.zeros(var0.shape) # Initialization
for ilev in range(var0.shape[1]):
    print('Interpolating level ',ilev,'out of ',var0.shape[1])
    interp_fx = interp1d(x=PERC_array[:,i0[var[:-4]]+ilev],y=quantile_array,bounds_error=False)
    output_percentile[:,ilev,:,:] = interp_fx(var0[:,ilev,:,:].values)

In [ ]:
output_percentile = output_percentile+0*var0**0

In [ ]:
output_percentile

In [ ]:
var0[:,ilev,:,:].values.max()

In [ ]:
PERC_array[:,i0[var[:-4]]+ilev].max()

In [ ]:
ilev = 29

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))
ax[0].hist(var0[:,ilev,:,:].values.flatten(),bins=100);
ax[1].hist(PERC_array[:,i0[var[:-4]]+ilev],bins=100);
ax[2].hist(output_percentile[:,ilev,:,:].values.flatten(),bins=100);

# Make Distributions Figure for final postdoc talks

## Font size

In [ ]:
fz = 40
lw = 6
siz = 100

In [ ]:
plt.rc('text', usetex=False)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
plt.rc('font', family='serif', size=fz)
mpl.rcParams['lines.linewidth'] = lw
plt.close('all')

## Load data and coordinates

In [ ]:
# Load coordinates (just pick any file from the climate model run)

# Comet path below
# coor = xr.open_dataset("/oasis/scratch/comet/ankitesh/temp_project/data/sp8fbp_minus4k.cam2.h1.0000-01-01-00000.nc",\
#                     decode_times=False)

# GP path below
path_0K = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/'
coor = xr.open_dataset(path_0K+"AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-09-02-00000.nc")

lat = coor.lat; lon = coor.lon; lev = coor.lev;
coor.close();

In [ ]:
# Comet path below
# TRAINDIR = '/oasis/scratch/comet/ankitesh/temp_project/PrepData/CRHData/'
# path = '/home/ankitesh/CBrain_project/CBRAIN-CAM/cbrain/'

# GP path below
TRAINDIR = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'
path = '/export/nfs0home/tbeucler/CBRAIN-CAM/cbrain/'
path_nnconfig = '/export/nfs0home/tbeucler/CBRAIN-CAM/nn_config/'

# Load hyam and hybm to calculate pressure field in SPCAM
path_hyam = 'hyam_hybm.pkl'
hf = open(path+path_hyam,'rb')
hyam,hybm = pickle.load(hf)

# Scale dictionary to convert the loss to W/m2
scale_dict = load_pickle(path_nnconfig+'scale_dicts/009_Wm2_scaling.pkl')

In [ ]:
coor.lev

In [ ]:
path_array = [
     '2021_03_18_O3_TRAIN_M4K_shuffle.nc',
#    '2021_03_18_O3_VALID_M4K.nc',
#    '2021_03_18_O3_TEST_P4K.nc',
#    '2021_01_24_O3_TRAIN_shuffle.nc',
#    '2021_03_18_O3_TEST_M4K.nc',
#    '2021_01_24_O3_VALID.nc',
#    '2021_01_24_O3_TEST.nc',
     '2021_03_18_O3_TRAIN_P4K_shuffle.nc',
#    '2021_03_18_O3_VALID_P4K.nc'
]

In [ ]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [ ]:
pathPKL = '/export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA'

In [ ]:
train_data = {}

In [ ]:
for ipath,path in enumerate(path_array):
    print('ipath=',ipath,'path=',path,'     ',end='\n')
    
    path_var = path_data + path 
    train_data[path] = xr.open_dataset(path_var)

In [ ]:
train_data

In [ ]:
import psutil
psutil.virtual_memory()

In [ ]:
136430485504/1e9

## Distribution of q vs RH at different levels

In [ ]:
def compute_bp(ds, var):
    """GCM state at beginning of time step before physics.
    ?BP = ?AP - physical tendency * dt

    Args:
        ds: entire xarray dataset
        var: BP variable name

    Returns:
        bp: xarray dataarray containing just BP variable, with the first time step cut.
    """
    base_var = var[:-2] + 'AP'
    return (ds[base_var] - ds[phy_dict[base_var]] * DT)[1:]  # Not the first time step

In [ ]:
def edg2bin(bin_edges):
    return 0.5*(bin_edges[1:]+bin_edges[:-1])

In [ ]:
def RH(T,qv,P0,PS,hyam,hybm):
    R = 287
    Rv = 461
    p = (hyam*P0+hybm*PS) # Total pressure (Pa)
    return Rv*p*qv/(R*esat(T))
def esat(T):
    T0 = 273.16
    T00 = 253.16
    omega = np.maximum(0,np.minimum(1,(T-T00)/(T0-T00)))

    return (T>T0)*eliq(T)+(T<T00)*eice(T)+(T<=T0)*(T>=T00)*(omega*eliq(T)+(1-omega)*eice(T))
def eliq(T):
    a_liq = np.array([-0.976195544e-15,-0.952447341e-13,0.640689451e-10,0.206739458e-7,0.302950461e-5,0.264847430e-3,0.142986287e-1,0.443987641,6.11239921]);
    c_liq = -80
    T0 = 273.16
    return 100*np.polyval(a_liq,np.maximum(c_liq,T-T0))
def eice(T):
    a_ice = np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,0.602588177e-7,0.615021634e-5,0.420895665e-3,0.188439774e-1,0.503160820,6.11147274]);
    c_ice = np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07])
    T0 = 273.16
    return (T>c_ice[0])*eliq(T)+\
(T<=c_ice[0])*(T>c_ice[1])*100*np.polyval(a_ice,T-T0)+\
(T<=c_ice[1])*100*(c_ice[3]+np.maximum(c_ice[2],T-T0)*(c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5]))


In [ ]:
train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,90]

In [ ]:
coor.lev[19]

In [ ]:
a = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,19]

In [ ]:
a = a.values

In [ ]:
histQm4K,edgm4K = np.histogram(a, bins=100, density=True)

In [ ]:
a = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,19]

In [ ]:
a = a.values

In [ ]:
histQp4K,edgp4K = np.histogram(a, bins=100, density=True)

In [ ]:
aT = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,49]
aq = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,19]
aPS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,90]

In [ ]:
print('T')
aT = aT.values
print('q')
aq = aq.values
print('PS')
aPS = aPS.values

In [ ]:
a = RH(aT,aq,P0,aPS,hyam[19],hybm[19])

In [ ]:
a.shape

In [ ]:
histRHm4K,edgm4K_RH = np.histogram(a, bins=100, density=True)

In [ ]:
aT = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,49]
aq = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,19]
aPS = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,90]

In [ ]:
print('T')
aT = aT.values
print('q')
aq = aq.values
print('PS')
aPS = aPS.values

In [ ]:
a = RH(aT,aq,P0,aPS,hyam[19],hybm[19])

In [ ]:
histRHP4K,edgP4K_RH = np.histogram(a, bins=100, density=True)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(15,15))

plt.subplots_adjust(wspace=None, hspace=-0.025)

# Top subplot = Specific humidity
ax[0].plot(edg2bin(edgm4K),histQm4K,color='b')
ax[0].plot(edg2bin(edgp4K),histQp4K,color='r')
ax[0].yaxis.tick_right()
#ax[0].set_xlabel('Specific humidity (kg kg$^{-1}$)')
ax[0].xaxis.tick_top()
ax[0].set_title('Specific humidity (kg kg$^{-1}$)')

# Bottom subplot = Relative humidity
ax[1].plot(edg2bin(edgm4K_RH),histRHm4K,color='b')
ax[1].plot(edg2bin(edgP4K_RH),histRHP4K,color='r')
ax[1].yaxis.tick_right()
ax[1].set_xlabel('Relative humidity')
#ax[1].set_title('PDF')

## Distribution of T vs BCONS at different levels

In [ ]:

def esat(T):
    T0 = 273.16
    T00 = 253.16
    omega = np.maximum(0,np.minimum(1,(T-T00)/(T0-T00)))

    return (T>T0)*eliq(T)+(T<T00)*eice(T)+(T<=T0)*(T>=T00)*(omega*eliq(T)+(1-omega)*eice(T))
def eliq(T):
    a_liq = np.array([-0.976195544e-15,-0.952447341e-13,0.640689451e-10,0.206739458e-7,0.302950461e-5,0.264847430e-3,0.142986287e-1,0.443987641,6.11239921]);
    c_liq = -80
    T0 = 273.16
    return 100*np.polyval(a_liq,np.maximum(c_liq,T-T0))
def eice(T):
    a_ice = np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,0.602588177e-7,0.615021634e-5,0.420895665e-3,0.188439774e-1,0.503160820,6.11147274]);
    c_ice = np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07])
    T0 = 273.16
    return (T>c_ice[0])*eliq(T)+\
(T<=c_ice[0])*(T>c_ice[1])*100*np.polyval(a_ice,T-T0)+\
(T<=c_ice[1])*100*(c_ice[3]+np.maximum(c_ice[2],T-T0)*(c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5]))

def qv(T,RH,P0,PS,hyam,hybm):
    R = 287
    Rv = 461
    p = (hyam*P0+hybm*PS) # Total pressure (Pa)

    return R*esat(T)*RH/(Rv*p)

def qsat(T,P0,PS,hyam,hybm):
    return qv(T,1,P0,PS,hyam,hybm)

def theta_e_calc(T,qv,P0,PS,hyam,hybm):
    p = (hyam*P0+hybm*PS) # Total pressure (Pa)

    tmelt  = 273.15
    CPD = 1005.7
    CPV = 1870.0
    CPVMCL = 2320.0
    RV = 461.5
    RD = 287.04
    EPS = RD/RV
    ALV0 = 2.501E6

    r = qv / (1. - qv)
    # get ev in hPa 
    ev_hPa = 100*p*r/(EPS+r)
    #get TL
    TL = (2840. / ((3.5*np.log(T)) - (np.log(ev_hPa)) - 4.805)) + 55.
    #calc chi_e:
    chi_e = 0.2854 * (1. - (0.28*r))
    P0_norm = (P0/(hyam*P0+hybm*PS))

    theta_e = T * P0_norm**chi_e * np.exp(((3.376/TL) - 0.00254) * r * 1000. * (1. + (0.81 * r)))

    return theta_e

def theta_e_sat_calc(T,qv,P0,PS,hyam,hybm):
    return theta_e_calc(T,qsat(T,P0,PS,hyam,hybm),P0,PS,hyam,hybm)

# TBP = compute_bp(ds,'TBP')
# QBP = compute_bp(ds,'QBP')

# return G*(theta_e_calc(TBP,QBP,ds['P0'],ds['PS'][1:,:,:],ds['hyam'],ds['hybm'])[:,-1,:,:]-\
#           theta_e_sat_calc(TBP,QBP,ds['P0'],ds['PS'][1:,:,:],ds['hyam'],ds['hybm']))/\
# theta_e_sat_calc(TBP,QBP,ds['P0'],ds['PS'][1:,:,:],ds['hyam'],ds['hybm'])   

In [ ]:
ilev = 23

In [ ]:
coor.lev[ilev]

In [ ]:
a = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,30+ilev]

In [ ]:
a = a.values

In [ ]:
histTm4K,edgm4K_T = np.histogram(a, bins=100, density=True)

In [ ]:
a = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,30+ilev]

In [ ]:
a = a.values

In [ ]:
histTp4K,edgp4K_T = np.histogram(a, bins=100, density=True)

In [ ]:
aT = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,30+ilev]
aq = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,ilev]
aPS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,90]

In [ ]:
print('T')
aT = aT.values
print('q')
aq = aq.values
print('PS')
aPS = aPS.values

In [ ]:
aTNS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,59]
aqNS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,29]

In [ ]:
print('T')
aTNS = aTNS.values
print('q')
aqNS = aqNS.values

In [ ]:
a = G*(theta_e_calc(aTNS,aqNS,P0,aPS,hyam[29],hybm[29])-\
   theta_e_sat_calc(aT,aq,P0,aPS,hyam[ilev],hybm[ilev]))/\
theta_e_sat_calc(aT,aq,P0,aPS,hyam[ilev],hybm[ilev])  

In [ ]:
histBm4K,edgm4K_B = np.histogram(a, bins=100, density=True)

In [ ]:
aT = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,30+ilev]
aq = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,ilev]
aPS = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,90]

In [ ]:

print('T')
aT = aT.values
print('q')
aq = aq.values
print('PS')
aPS = aPS.values

In [ ]:
aTNS = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,59]
aqNS = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,29]

In [ ]:
print('T')
aTNS = aTNS.values
print('q')
aqNS = aqNS.values

In [ ]:
a = G*(theta_e_calc(aTNS,aqNS,P0,aPS,hyam[29],hybm[29])-\
   theta_e_sat_calc(aT,aq,P0,aPS,hyam[ilev],hybm[ilev]))/\
theta_e_sat_calc(aT,aq,P0,aPS,hyam[ilev],hybm[ilev])  

In [ ]:
histBp4K,edgp4K_B = np.histogram(a, bins=100, density=True)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(15,15))

plt.subplots_adjust(wspace=None, hspace=-0.025)

# Top subplot = Specific humidity
ax[0].plot(edg2bin(edgm4K_T),histTm4K,color='b')
ax[0].plot(edg2bin(edgp4K_T),histTp4K,color='r')
ax[0].set_title('Temperature (K)')
ax[0].yaxis.tick_right()
ax[0].xaxis.tick_top()

# Bottom subplot = Relative humidity
ax[1].plot(edg2bin(edgm4K_B),histBm4K,color='b')
ax[1].plot(edg2bin(edgp4K_B),histBp4K,color='r')
ax[1].set_xlabel('Buoyancy (m s$^{-2}$)')
ax[1].yaxis.tick_right()

## Distribution of T vs BMSE at different levels

In [ ]:
def esat(T):
    T0 = 273.16
    T00 = 253.16
    omega = np.maximum(0,np.minimum(1,(T-T00)/(T0-T00)))

    return (T>T0)*eliq(T)+(T<T00)*eice(T)+(T<=T0)*(T>=T00)*(omega*eliq(T)+(1-omega)*eice(T))
def eliq(T):
    a_liq = np.array([-0.976195544e-15,-0.952447341e-13,0.640689451e-10,0.206739458e-7,0.302950461e-5,0.264847430e-3,0.142986287e-1,0.443987641,6.11239921]);
    c_liq = -80
    T0 = 273.16
    return 100*np.polyval(a_liq,np.maximum(c_liq,T-T0))
def eice(T):
    a_ice = np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,0.602588177e-7,0.615021634e-5,0.420895665e-3,0.188439774e-1,0.503160820,6.11147274]);
    c_ice = np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07])
    T0 = 273.16
    return (T>c_ice[0])*eliq(T)+\
(T<=c_ice[0])*(T>c_ice[1])*100*np.polyval(a_ice,T-T0)+\
(T<=c_ice[1])*100*(c_ice[3]+np.maximum(c_ice[2],T-T0)*(c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5]))

def qv(T,RH,P0,PS,hyam,hybm):
    R = 287
    Rv = 461
#     p = P0 * hyam + PS * hybm
#     p = p.astype(np.float32)
    S = T.shape;
    #print(S)
    p = np.moveaxis(np.tile(P0,(30,1)),[0,1],[1,0]) *\
    np.moveaxis(np.tile(hyam,(1,1)),[0,1],[0,1]) + \
    np.moveaxis(np.tile(PS,(30,1)),0,1) * \
    np.moveaxis(np.tile(hybm,(1,1)),[0,1],[0,1])

    return R*esat(T)*RH/(Rv*p)

def qsat(T,P0,PS,hyam,hybm):
    return qv(T,1,P0,PS,hyam,hybm)

def bmse_calc(T,qv,P0,PS,hyam,hybm):
    eps = 0.622 # Ratio of molecular weight(H2O)/molecular weight(dry air)
    R_D = 287 # Specific gas constant of dry air in J/K/kg
    Rv = 461
    # Calculate kappa
    QSAT0 = qsat(T,P0,PS,hyam,hybm)
    kappa = (1+(L_V**2)*QSAT0/(Rv*C_P*(T**2))).astype(np.float32)
    # Calculate geopotential
    r = qv/(qv**0-qv)
    Tv = T*(r**0+r/eps)/(r**0+r)
#     p = P0 * hyam + PS * hybm
#     p = p.astype(np.float32)
#     p = (hyam*P0+hybm*PS).values # Total pressure (Pa)
    S = Tv.shape;
    p = np.moveaxis(np.tile(P0,(30,1)),[0,1],[1,0]) *\
    np.moveaxis(np.tile(hyam,(1,1)),[0,1],[0,1]) + \
    np.moveaxis(np.tile(PS,(30,1)),0,1) * \
    np.moveaxis(np.tile(hybm,(1,1)),[0,1],[0,1])
    # Geopotential calculation below
    RHO = p/(R_D*Tv)
    Z = -sin.cumtrapz(x=p,y=1/(G*RHO),axis=1)
    Z = np.concatenate((0*Z[:,0:1]**0,Z),axis=1)
    # Assuming near-surface is at 2 meters
    Z = (Z-Z[:,[29]])+2 
    # Calculate MSEs of plume and environment
    Tile_dim = [1,30]
    h_plume = (np.tile(np.expand_dims(C_P*T[:,-1]+L_V*qv[:,-1],axis=1),Tile_dim)).astype(np.float32)
    h_satenv = (C_P*T+L_V*qv+G*Z).astype(np.float32)
    return (G/kappa)*(h_plume-h_satenv)/(C_P*T)

# TBP = compute_bp(ds,'TBP')
# QBP = compute_bp(ds,'QBP')

# return G*(theta_e_calc(TBP,QBP,ds['P0'],ds['PS'][1:,:,:],ds['hyam'],ds['hybm'])[:,-1,:,:]-\
#           theta_e_sat_calc(TBP,QBP,ds['P0'],ds['PS'][1:,:,:],ds['hyam'],ds['hybm']))/\
# theta_e_sat_calc(TBP,QBP,ds['P0'],ds['PS'][1:,:,:],ds['hyam'],ds['hybm'])   

In [ ]:
ilev = 23

In [ ]:
coor.lev[ilev]

In [ ]:
a = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,30+ilev]

In [ ]:
a = a.values

In [ ]:
histTm4K,edgm4K_T = np.histogram(a, bins=100, density=True)

In [ ]:
a = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,30+ilev]

In [ ]:
a = a.values

In [ ]:
histTp4K,edgp4K_T = np.histogram(a, bins=100, density=True)

In [ ]:
aT = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,30:60]
aq = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,:30]
aPS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,90]

In [ ]:
print('T')
aT = aT.values
print('q')
aq = aq.values
print('PS')
aPS = aPS.values

In [ ]:
path_dm4K = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/sp8fbp_minus4k/sp8fbp_minus4k.cam2.h2.'
ds = xr.open_mfdataset(path_dm4K+'0001-01-05-00000.nc',\
                          decode_times=False)

In [ ]:
train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'].shape

In [ ]:
47177728/(472*1e6)

In [ ]:
b_mse = np.zeros((47177728, 30))

In [ ]:
# Calculate bmse in small batches
for ibatch in range(471):
    print('ibatch=',ibatch,'             ',end='\r')
    aT = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][int(ibatch*1e5):int((ibatch+1)*1e5),30:60].values;
    aq = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][int(ibatch*1e5):int((ibatch+1)*1e5),:30].values;
    aPS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][int(ibatch*1e5):int((ibatch+1)*1e5),90].values;
    print('Done loading the arrays       ',end='\r')
    a = bmse_calc(aT,aq,P0,aPS,hyam,hybm);
    b_mse[int(ibatch*1e5):int((ibatch+1)*1e5),:] = a;
print('Final index')
aT = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][int((ibatch+1)*1e5):,30:60];
aq = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][int((ibatch+1)*1e5):,:30];
aPS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][int((ibatch+1)*1e5):,90];
print('Done loading the arrays       ',end='\r')
a = bmse_calc(aT,aq,P0,aPS,hyam,hybm)
b_mse[int((ibatch+1)*1e5):,:] = a

In [ ]:
# a_notload = bmse_calc(train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,30:60],
#                       train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,:30],
#                       P0,
#                       train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,90],
#                       coor['hyam'],coor['hybm'])

In [ ]:
# a = bmse_calc(aT,aq,P0,aPS,hyam,hybm)

In [ ]:
histBm4K,edgm4K_B = np.histogram(b_mse[:,ilev], bins=100, density=True)

In [ ]:
b_mse = np.zeros((47177728, 30))

In [ ]:
train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'].shape

In [ ]:
# Calculate bmse in small batches
for ibatch in range(471):
    print('ibatch=',ibatch,'             ',end='\r')
    aT = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][int(ibatch*1e5):int((ibatch+1)*1e5),30:60].values;
    aq = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][int(ibatch*1e5):int((ibatch+1)*1e5),:30].values;
    aPS = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][int(ibatch*1e5):int((ibatch+1)*1e5),90].values;
    print('Done loading the arrays       ',end='\r')
    a = bmse_calc(aT,aq,P0,aPS,hyam,hybm);
    b_mse[int(ibatch*1e5):int((ibatch+1)*1e5),:] = a;
print('Final index')
aT = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][int((ibatch+1)*1e5):,30:60];
aq = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][int((ibatch+1)*1e5):,:30];
aPS = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][int((ibatch+1)*1e5):,90];
print('Done loading the arrays       ',end='\r')
a = bmse_calc(aT,aq,P0,aPS,hyam,hybm)
b_mse[int((ibatch+1)*1e5):,:] = a

In [ ]:
aPS

In [ ]:
bmse_calc(aT,aq,P0,aPS,hyam,hybm)

In [ ]:
b_mse[int((ibatch+1)*1e5):,:]

In [ ]:
histBp4K,edgp4K_B = np.histogram(a[ilev], bins=100, density=True)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(15,15))

plt.subplots_adjust(wspace=None, hspace=-0.025)

# Top subplot = Specific humidity
# ax[0].plot(edg2bin(edgm4K_T),histTm4K,color='b')
# ax[0].plot(edg2bin(edgp4K_T),histTp4K,color='r')
# ax[0].set_title('Temperature (K)')
# ax[0].yaxis.tick_right()
# ax[0].xaxis.tick_top()

# Bottom subplot = Relative humidity
ax[1].plot(edg2bin(edgm4K_B),histBm4K,color='b')
ax[1].plot(edg2bin(edgp4K_B),histBp4K,color='r')
ax[1].set_xlabel('Buoyancy (m s$^{-2}$)')
ax[1].yaxis.tick_right()

In [ ]:
F_data = {'edgm4K_B':edgm4K_B,'histBm4K':histBm4K,'edgp4K_B':edgp4K_B,'histBp4K':histBp4K}

In [ ]:
path = '2021_06_17_distributions_BMSE'

hf = open(pathPKL+'/'+path+'.pkl','rb')

test = pickle.load(hf)
hf.close()
print('\n')

## Distribution of LHF vs LHF/nsDELQ 

In [ ]:
train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,93]

In [ ]:
aLHF = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,93]

In [ ]:
aLHF = aLHF.values

In [ ]:
histLHFm4K,edgm4K_LHF = np.histogram(aLHF, bins=100, density=True)

In [ ]:
aLHF = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,93]

In [ ]:
aLHF = aLHF.values

In [ ]:
histLHFp4K,edgp4K_LHF = np.histogram(aLHF, bins=100, density=True)

In [ ]:
def esat(T):
    T0 = 273.16
    T00 = 253.16
    omega = np.maximum(0,np.minimum(1,(T-T00)/(T0-T00)))

    return (T>T0)*eliq(T)+(T<T00)*eice(T)+(T<=T0)*(T>=T00)*(omega*eliq(T)+(1-omega)*eice(T))

def eliq(T):
    a_liq = np.array([-0.976195544e-15,-0.952447341e-13,0.640689451e-10,0.206739458e-7,0.302950461e-5,0.264847430e-3,0.142986287e-1,0.443987641,6.11239921]);
    c_liq = -80
    T0 = 273.16
    return 100*np.polyval(a_liq,np.maximum(c_liq,T-T0))

def eice(T):
    a_ice = np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,0.602588177e-7,0.615021634e-5,0.420895665e-3,0.188439774e-1,0.503160820,6.11147274]);
    c_ice = np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07])
    T0 = 273.16
    return (T>c_ice[0])*eliq(T)+\
(T<=c_ice[0])*(T>c_ice[1])*100*np.polyval(a_ice,T-T0)+\
(T<=c_ice[1])*100*(c_ice[3]+np.maximum(c_ice[2],T-T0)*(c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5]))

def qv(T,RH,P0,PS,hyam,hybm):
    R = 287
    Rv = 461
    p = (hyam*P0+hybm*PS) # Total pressure (Pa)

    return R*esat(T)*RH/(Rv*p)

def qsat(T,P0,PS,hyam,hybm):
    return qv(T,1,P0,PS,hyam,hybm)

# QBP = compute_bp(ds,'QBP')
# Qden = qsat(ds['TS'][1:,:,:],ds['P0'],ds['PS'][1:,:,:],ds['hyam'][:,-1],ds['hybm'][:,-1])-QBP[:,-1,:,:].values
# return ds['LHFLX'][:-1]/(L_V*np.maximum(eps,Qden))

In [ ]:
aTNS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,59]
aqNS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,29]
aPS = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,90]
aLHF = train_data['2021_03_18_O3_TRAIN_M4K_shuffle.nc']['vars'][:,93]

In [ ]:
print('T')
aTNS = aTNS.values
print('q')
aqNS = aqNS.values
print('PS')
aPS = aPS.values
print('LHF')
aLHF = aLHF.values

In [ ]:
a = aLHF/(L_V*np.maximum(1e-3,(qsat(aTNS,P0,aPS,hyam[-1],hybm[-1])-aqNS)))

In [ ]:
histLHFnm4K,edgm4K_LHFn = np.histogram(a, bins=100, density=True)

In [ ]:
aTNS = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,59]
aqNS = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,29]
aPS = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,90]
aLHF = train_data['2021_03_18_O3_TRAIN_P4K_shuffle.nc']['vars'][:,93]

In [ ]:
print('T')
aTNS = aTNS.values
print('q')
aqNS = aqNS.values
print('PS')
aPS = aPS.values
print('LHF')
aLHF = aLHF.values

In [ ]:
a = aLHF/(L_V*np.maximum(1e-3,(qsat(aTNS,P0,aPS,hyam[-1],hybm[-1])-aqNS)))

In [ ]:
histLHFnp4K,edgp4K_LHFn = np.histogram(a, bins=100, density=True)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(15,15))

plt.subplots_adjust(wspace=None, hspace=-0.025)

# Top subplot = Specific humidity
ax[0].plot(edg2bin(edgm4K_LHF),np.log10(histLHFm4K),color='b')
ax[0].plot(edg2bin(edgp4K_LHF),np.log10(histLHFp4K),color='r')
ax[0].set_title('Latent Heat Flux (W m$^{-2}$)')
ax[0].yaxis.tick_right()
ax[0].xaxis.tick_top()
#ax[0].set_title('PDF')

#Bottom subplot = Relative humidity
ax[1].plot(edg2bin(test['edgm4K_LHFn']),np.log10(test['histLHFnm4K']),color='b')
ax[1].plot(edg2bin(test['edgp4K_LHFn']),np.log10(test['histLHFnp4K']),color='r')
ax[1].set_xlabel('Normalized LHF (kg m$^{-2}$ s$^{-1}$)')
ax[1].yaxis.tick_right()
ax[1].set_title('PDF')

## Save all histograms

In [ ]:
F_data = {'edgm4K':edgm4K,'histQm4K':histQm4K,'edgp4K':edgp4K,'histQp4K':histQp4K,'edgm4K_RH':edgm4K_RH,
          'histRHm4K':histRHm4K,'edgP4K_RH':edgP4K_RH,'histRHP4K':histRHP4K,'edgm4K_T':edgm4K_T,
          'histTm4K':histTm4K,'edgp4K_T':edgp4K_T,'histTp4K':histTp4K,'edgm4K_B':edgm4K_B,
          'histBm4K':histBm4K,'edgp4K_B':edgp4K_B,'histBp4K':histBp4K,'edgm4K_LHF':edgm4K_LHF,
          'histTm4K':histTm4K,'edgp4K_LHF':edgp4K_LHF,'histTp4K':histTp4K,'edgm4K_LHFn':edgm4K_LHFn,
          'histLHFnm4K':histLHFnm4K,'edgp4K_LHFn':edgp4K_LHFn,'histLHFnp4K':histLHFnp4K}

In [ ]:
path = '2021_06_17_distributions_RH_BMSE_LHFn'

hf = open(pathPKL+'/'+path+'.pkl','rb')

test = pickle.load(hf)
hf.close()
print('\n')

# Make loss vs time figure for final postdoc talks

## Preliminary data analysis

In [ ]:
path_PKL = '/export/nfs0home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/'

In [ ]:
path_MLR_BF = path_PKL + 'PKL_DATA2021_04_26_MLR_hist.pkl'
hf = open(path_MLR_BF,'rb')
hist_MLR_BF = pickle.load(hf)
hist_MLR_BF = hist_MLR_BF['hist']

In [ ]:
path_NN_BF1 = path_PKL + 'PKL_DATA2021_04_26_NN_hist.pkl'
path_NN_BF2 = path_PKL + 'PKL_DATA2021_05_04_NN_hist.pkl'

hf = open(path_NN_BF1,'rb')
hist_NN_BF1 = pickle.load(hf)
hist_NN_BF1 = hist_NN_BF1['hist']

hf = open(path_NN_BF2,'rb')
hist_NN_BF2 = pickle.load(hf)
hist_NN_BF2 = hist_NN_BF2['hist']

In [ ]:
path_MLR_CI = path_PKL + 'PKL_DATA2021_04_26_MLR_RH_BCONS_LHF_nsDELQ_hist.pkl'
hf = open(path_MLR_CI,'rb')
hist_MLR_CI = pickle.load(hf)
hist_MLR_CI = hist_MLR_CI['hist']

In [ ]:
path_MLR_TfromNS = path_PKL + 'PKL_DATA2021_04_26_MLR_RH_TfromNS_LHF_nsDELQ_hist.pkl'
hf = open(path_MLR_TfromNS,'rb')
hist_MLR_TfromNS = pickle.load(hf)
hist_MLR_TfromNS = hist_MLR_TfromNS['hist']

In [ ]:
path_NN_TfromNS = path_PKL + 'PKL_DATA2021_04_26_NN_RH_TfromNS_LHF_nsDELQ_hist.pkl'
hf = open(path_NN_TfromNS,'rb')
hist_NN_TfromNS = pickle.load(hf)
hist_NN_TfromNS = hist_NN_TfromNS['hist']

In [ ]:
path_NN_BCONS1 = path_PKL + 'PKL_DATA2021_05_05_NN_RH_BCONS_LHF_nsDELQ_hist10.pkl'
hf = open(path_NN_BCONS1,'rb')
hist_NN_BCONS1 = pickle.load(hf)
hist_NN_BCONS1 = hist_NN_BCONS1['hist']

path_NN_BCONS2 = path_PKL + 'PKL_DATA2021_05_05_NN_RH_BCONS_LHF_nsDELQ_hist20.pkl'
hf = open(path_NN_BCONS2,'rb')
hist_NN_BCONS2 = pickle.load(hf)
hist_NN_BCONS2 = hist_NN_BCONS2['hist']

hist_NN_BCONS0 = {}
for key in hist_NN_BCONS1.keys():
    hist_NN_BCONS0[key] = np.append(hist_NN_BCONS1[key],hist_NN_BCONS2[key])

In [ ]:
fz = 15
lw = 4

In [ ]:
plt.rc('text', usetex=False)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
plt.rc('font', family='serif', size=fz)
mpl.rcParams['lines.linewidth'] = lw
plt.close('all')

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(15,10))

for iax in range(6):
    
    iax_ar = iax//3
    subp = ax[iax_ar][iax-3*iax_ar]
    
    if iax==0: h = hist_MLR_BF; tit = 'MLR BF'
    elif iax==1: h = hist_NN_BF1; tit = 'NN BF 04 26'
    elif iax==2: h = hist_MLR_TfromNS; tit = 'MLR TfromNS'
    elif iax==3: h = hist_MLR_CI; tit = 'MLR CI'
    elif iax==4: h = hist_NN_TfromNS; tit = 'NN TfromNS'
    elif iax==5: h = hist_NN_BCONS0; tit = 'NN BCONS'
    
    for key in h.keys():
        subp.plot(h[key],label=key)
    subp.legend()
    subp.set_title(tit+' '+str(h['val_loss'][-1])[:4])
    subp.grid()

In [ ]:
path_LOGI_TfromNS = path_PKL + 'PKL_DATA2021_04_26_LOGI_PERC_RH_TfromNS_LHF_nsDELQ_hist.pkl'
hf = open(path_LOGI_TfromNS,'rb')
hist_LOGI_TfromNS = pickle.load(hf)
hist_LOGI_TfromNS = hist_LOGI_TfromNS['hist']

In [ ]:
path_LOGI_BCONS = path_PKL + 'PKL_DATA2021_04_26_LOGI_PERC_RH_BCONS_LHF_nsDELQ_hist.pkl'
hf = open(path_LOGI_BCONS,'rb')
hist_LOGI_BCONS = pickle.load(hf)
hist_LOGI_BCONS = hist_LOGI_BCONS['hist']

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(15,10))

for iax in range(2):
    
    iax_ar = iax//1
    subp = ax[iax]
    
    if iax==0: h = hist_LOGI_TfromNS; tit = 'LOGI TfromNS'
    elif iax==1: h = hist_LOGI_BCONS; tit = 'LOGI BCONS'
    
    for key in h.keys():
        subp.plot(h[key],label=key)
    subp.legend()
    subp.set_title(tit+' '+str(h['val_loss'][-1])[:6])
    subp.grid()

## Figure

In [ ]:
fz = 40
lw = 6
siz = 250

plt.rc('text', usetex=False)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
plt.rc('font', family='serif', size=fz)
mpl.rcParams['lines.linewidth'] = lw
plt.close('all')

In [ ]:
epoch = np.arange(1,21)

In [ ]:
plt.figure(figsize=(20,10))

plt.scatter(epoch,hist_NN_BF1['val_loss'],s=siz,color='b')
plt.plot(epoch,hist_NN_BF1['val_loss'],color='b')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss (W$^{2}$ m${-4})$')
plt.xlim((0,20))

In [ ]:
plt.figure(figsize=(20,10))

plt.scatter(epoch,hist_NN_BF1['val_loss'],s=siz,color='b')
plt.plot(epoch,hist_NN_BF1['val_loss'],color='b')
plt.scatter(epoch,hist_NN_BF1['trainM4K_RG_loss'],s=siz,color='g')
plt.plot(epoch,hist_NN_BF1['trainM4K_RG_loss'],color='g')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss (W$^{2}$ m${-4})$')
plt.xlim((0,20))

In [ ]:
plt.figure(figsize=(20,10))

plt.scatter(epoch,hist_NN_BF1['val_loss'],s=siz,color='b')
plt.plot(epoch,hist_NN_BF1['val_loss'],color='b')

plt.scatter(epoch,hist_MLR_BF['val_loss'],s=siz,color='b',marker='s')
plt.plot(epoch,hist_MLR_BF['val_loss'],color='b')

plt.xlabel('Number of Epochs')
plt.ylabel('Loss (W$^{2}$ m${-4})$')
plt.xlim((0,20))

In [ ]:
plt.figure(figsize=(20,10))

plt.scatter(epoch,hist_NN_BF1['val_loss'],s=siz,color='b')
plt.plot(epoch,hist_NN_BF1['val_loss'],color='b')

plt.scatter(epoch,hist_MLR_BF['val_loss'],s=siz,color='b',marker='s')
plt.plot(epoch,hist_MLR_BF['val_loss'],color='b')

plt.scatter(epoch,hist_MLR_BF['trainM4K_RG_loss'],s=siz,color='g',marker='s')
plt.plot(epoch,hist_MLR_BF['trainM4K_RG_loss'],color='g',marker='s')

plt.xlabel('Number of Epochs')
plt.ylabel('Loss (W$^{2}$ m${-4})$')
plt.xlim((0,20))

In [ ]:
plt.figure(figsize=(20,10))

plt.scatter(epoch,hist_NN_BF1['val_loss'],s=siz,color='b')
plt.plot(epoch,hist_NN_BF1['val_loss'],color='b')

# plt.scatter(epoch,hist_MLR_BF['val_loss'],s=siz,color='b',marker='s')
# plt.plot(epoch,hist_MLR_BF['val_loss'],color='b')

plt.scatter(epoch,hist_MLR_BF['trainM4K_RG_loss'],s=siz,color='g',marker='s')
plt.plot(epoch,hist_MLR_BF['trainM4K_RG_loss'],color='g',marker='s')

plt.scatter(epoch,hist_MLR_CI['val_loss'],color='b',marker='*',s=2*siz)
plt.plot(epoch,hist_MLR_CI['val_loss'],color='b',marker='*')

plt.scatter(epoch,hist_MLR_CI['trainM4K_RG_loss'],color='g',marker='*',s=2*siz)
plt.plot(epoch,hist_MLR_CI['trainM4K_RG_loss'],color='g',marker='*')

plt.xlabel('Number of Epochs')
plt.ylabel('Loss (W$^{2}$ m${-4})$')
plt.xlim((0,20))

In [ ]:
plt.figure(figsize=(20,10))

# plt.scatter(epoch,hist_NN_BF1['trainP4K_loss'],s=siz,color='r')
# plt.plot(epoch,hist_NN_BF1['trainP4K_loss'],color='r')

# plt.scatter(epoch,hist_MLR_BF['val_loss'],s=siz,color='b',marker='s')
# plt.plot(epoch,hist_MLR_BF['val_loss'],color='b')

plt.scatter(epoch,hist_MLR_BF['trainP4K_loss'],s=siz,color='r',marker='s')
plt.plot(epoch,hist_MLR_BF['trainP4K_loss'],color='r',marker='s')

plt.scatter(epoch,hist_MLR_CI['trainP4K_loss'],color='r',marker='*',s=2*siz)
plt.plot(epoch,hist_MLR_CI['trainP4K_loss'],color='r',marker='*')

# plt.scatter(epoch,hist_MLR_CI['trainM4K_RG_loss'],color='g',marker='*',s=2*siz)
# plt.plot(epoch,hist_MLR_CI['trainM4K_RG_loss'],color='g',marker='*')

plt.xlabel('Number of Epochs')
plt.ylabel('Loss (W$^{2}$ m${-4})$')
plt.xlim((0,20))